In [2]:
from vizdoom import *
import random 
import time

In [ ]:
from vizdoom import *
import random
import time

game = DoomGame()
game.load_config("vizdoom/scenarios/basic.cfg")
game.init()
game.set_render_crosshair(True)

shoot = [0, 0, 1]
left = [1, 0, 0]
right = [0, 1, 0]
actions = [shoot, left, right]

episodes = 10
for i in range(episodes):
    game.new_episode()
    while not game.is_episode_finished():
        state = game.get_state()
        img = state.screen_buffer
        misc = state.game_variables
        reward = game.make_action(random.choice(actions))
        print( "\treward:", reward)
        time.sleep(0.02)
    print ("Result:", game.get_total_reward())
    time.sleep(2)

	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: 100.0
	reward: -1.0
Result: 94.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: 100.0
	reward: -1.0
Result: 92.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -6.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -6.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -6.0
	rewar

## Going to use OpenAI Gym for this
There is a library already for wrapping VizDoom in Gym. However, I want to learn how to wrap environments in OpenAI Gym myself for the future
 

In [3]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import cv2
import numpy as np
from vizdoom import *
import random 
import time

In [42]:
#creating vizdoom class
class ViZDoomGame(Env):
    def __init__(self,render = False):
        super().__init__()
        self.game = DoomGame()
        self.game.load_config("vizdoom/scenarios/basic.cfg")
       
        
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        self.game.init()

        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8) # 240 , 320 , 3
        self.action_space = Discrete(3)
    def step(self, action):
        #return super().step(action)
        actions = np.identity(3) # identity matrix perfectly aligns with the shoot left right arrays
        reward = self.game.make_action(actions[action],4)
        
        
        if self.game.get_state(): 
            state = self.game.get_state().screen_buffer
           
            ammo = self.game.get_state().game_variables[0]
            info = ammo
        else: 
            state = np.zeros(self.observation_space.shape)
            info = 0 
        
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info 
        
        
    def reset(self):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
    def close(self):
    #    return super().close()
        self.game.close()
    
    def render(self):
        #return super().render()
        pass 
    

In [49]:
from vizdoom import *
import gym
from gym import spaces
import numpy as np

class VizDoomEnv(gym.Env):
    def __init__(self, config_file):
        super(VizDoomEnv, self).__init__()
        self.game = DoomGame()
        self.game.load_config(config_file)
        self.game.set_window_visible(True)
        self.game.set_mode(Mode.PLAYER)
        self.game.init()

        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(3)  #TURN_LEFT, TURN_RIGHT, ATTACK
        # Example for using image as input:
        self.observation_space = spaces.Box(low=0, high=255, shape=(240, 320, 3), dtype=np.uint8)

    def step(self, action):
        self.game.make_action(action)
        state = self.game.get_state()
        reward = self.game.get_last_reward()
        done = self.game.is_episode_finished()
        info = {}
        if state is not None and state.screen_buffer is not None:
            observation = state.screen_buffer.transpose(1, 2, 0)  # Transpose to (240, 320, 3)
        else:
            observation = np.zeros(self.observation_space.shape, dtype=np.uint8)
        terminated = done
        truncated = False  # Set this to True if you have a condition for truncation
        return observation, reward, terminated, truncated, info

    def reset(self, **kwargs):
        seed = kwargs.get('seed', None)
        if seed is not None:
            super().reset(seed=seed)
        self.game.new_episode()
        state = self.game.get_state()
        if state is not None and state.screen_buffer is not None:
            observation = state.screen_buffer.transpose(1, 2, 0)  # Transpose to (240, 320, 3)
        else:
            observation = np.zeros(self.observation_space.shape, dtype=np.uint8)
        return observation, {}

    def render(self, mode='human'):
        pass

    def close(self):
        self.game.close()

# Create the environment
env = VizDoomEnv("ViZDoom/scenarios/basic.cfg")

# Check the environment
from stable_baselines3.common.env_checker import check_env
check_env(env)

AssertionError: Your environment must inherit from the gymnasium.Env class cf. https://gymnasium.farama.org/api/env/

In [30]:
# test
env = ViZDoomGame(render=True)

In [26]:
state = env.reset()

In [28]:
env.step(0)

(array([[[67, 67, 67, ..., 67, 67, 67],
         [91, 91, 79, ..., 91, 79, 79],
         [67, 67, 67, ..., 67, 67, 67],
         ...,
         [19, 19, 11, ..., 47, 47, 55],
         [19, 27, 19, ..., 47, 47, 47],
         [11, 19, 19, ..., 27, 19, 19]],
 
        [[67, 67, 67, ..., 67, 67, 67],
         [91, 91, 79, ..., 91, 79, 79],
         [67, 67, 67, ..., 67, 67, 67],
         ...,
         [19, 19, 11, ..., 47, 47, 55],
         [19, 27, 19, ..., 47, 47, 47],
         [11, 19, 19, ..., 27, 19, 19]],
 
        [[67, 67, 67, ..., 67, 67, 67],
         [91, 91, 79, ..., 91, 79, 79],
         [67, 67, 67, ..., 67, 67, 67],
         ...,
         [19, 19, 11, ..., 47, 47, 55],
         [19, 27, 19, ..., 47, 47, 47],
         [11, 19, 19, ..., 27, 19, 19]]], dtype=uint8),
 -4.0,
 False,
 array([50.]))

In [30]:
env.step(2)

(array([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]),
 99.0,
 True,
 0)

In [31]:
env.close()

### Environment done, can train now

In [43]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common import env_checker

In [6]:
logs = "./logs"

In [44]:
env = ViZDoomGame()


In [45]:
env_checker.check_env(env)

TypeError: The reset() method must accept a `seed` parameter

In [46]:
model = PPO("CnnPolicy",env,tensorboard_log=logs,device="cpu",verbose=1,learning_rate=0.0001,n_steps=256)
# stable baselines documentation says that ppo runs better on cpu

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [47]:
model.learn(total_timesteps=100_000)

TypeError: ViZDoomGame.reset() got an unexpected keyword argument 'seed'